# PINECONEを用いたvectorサーチのデモ

1. 技術ブログの検索
  - 決まったフォーマットで出力
  - URLも同時に返却
  - 上位3つくらい

2. 技術ブログの登録

- https://gpt-index.readthedocs.io/en/v0.5.27/how_to/integrations/vector_stores.html


# 方針

- LangChain.Agentを利用せずに、LLamaIndex単体で実装

In [3]:
import os

from dotenv import load_dotenv
from llama_index import SimpleWebPageReader, LLMPredictor, ServiceContext, GPTPineconeIndex, OpenAIEmbedding
from llama_index.prompts.prompts import QuestionAnswerPrompt, RefinePrompt
import pinecone
from langchain import OpenAI

INDEX_NAME = "chatgpt-search-index"

PREDICTOR_MODEL_NAME = "gpt-3.5-turbo"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

INITIAL_URLS = [
    "https://dev.classmethod.jp/articles/lang-chain-agent-customized-by-llama-index-tool/",
    "https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155",
    "https://runble1.com/gcp-terraform-cloud-run/"
]
ADDITIONAL_URL = "https://zenn.dev/tfutada/articles/acf8adbb2ba5be"

# カスタムテンプレートの作成
CUSTOM_TEXT_QA_PROMPT_TMPL = (
    "コンテキストは以下です. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "コンテキストが与えられた場合, "
    "質問に回答してください: {query_str}\n"
)
CUSTOM_TEXT_QA_PROMPT = QuestionAnswerPrompt(CUSTOM_TEXT_QA_PROMPT_TMPL)

CUSTOM_REFINE_PROMPT_TMPL = (
    "元の質問: {query_str}\n"
    "オリジナルの回答: {existing_answer}\n"
    "以下のコンテキストを使って、オリジナルの回答を推敲することができます.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "コンテキストを元に、オリジナルの回答を、より元の質問に沿ったものに推敲してください. "
    "もしコンテキストが有用なものでなければ、オリジナルの回答を返却してください"
)
CUSTOM_REFINE_PROMPT = RefinePrompt(CUSTOM_REFINE_PROMPT_TMPL)

# 環境変数の読み込み
load_dotenv('../.env')

True

In [ ]:
# デバッグする際に実行
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Indexの構築 & 検索

In [11]:
# clientの作成
pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENVIRONMENT"])
pinecone.create_index(
    INDEX_NAME,
    dimension=1536, 
    metric="euclidean", 
    pod_type="p1"
)
pinecone_index = pinecone.Index(INDEX_NAME)

In [6]:
# webページからdocumentクラスを作成
documents = SimpleWebPageReader(html_to_text=True).load_data(INITIAL_URLS)

# extra_infoにurlを追加
for document, url in zip(documents, INITIAL_URLS):
    document.extra_info = {"url": url}

In [13]:
# 既にDBにdocumentが存在する場合、その内容を利用
if pinecone_index.describe_index_stats().total_vector_count > 0:
    initial_documents = []
else:
    initial_documents = documents

llm_predictor = LLMPredictor(
    llm=OpenAI(
        temperature=0, model_name=PREDICTOR_MODEL_NAME
    )
)

embed_model = OpenAIEmbedding(
    model=EMBEDDING_MODEL_NAME
)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model
)

index = GPTPineconeIndex.from_documents(
    pinecone_index=pinecone_index,
    documents=initial_documents, service_context=service_context
)

/Users/ikegamikenshin/Documents/study/chatgpt-practice/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:158: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/ikegamikenshin/Documents/study/chatgpt-practice/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:661: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 29979 tokens


In [14]:
search_query = "calibrationってどんな技術だっけ？"

response = index.query(
    search_query,
    similarity_top_k=3,
    text_qa_template=CUSTOM_TEXT_QA_PROMPT,
    refine_template=CUSTOM_REFINE_PROMPT
)

print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 17252 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 21 tokens


response: Calibrationは、測定器やモデルの出力値を正確に調整するための技術であり、信頼性を高めることを指します。機械学習においては、モデルの予測値と実際の値の分布が異なる場合に、モデルの信頼性を高めるために必要です。具体的には、確率予測の信頼度を高めるために、Sigmoid/Platt ScaleやIsotonic Regressionなどの手法があります。Isotonic Regressionは、単調非減少な回帰曲線を学習し、確率予測の信頼性を高める手法です。また、Calibration Curveを使って確率予測の信頼度を可視化することもできます。Calibrationの評価指標としては、Brier Scoreがよく使われます。Brier Scoreは、確率予測した値とラベル(0,1)との二乗誤差の平均を表します。ただし、LightGBMや最近のNNは自信過剰で、Calibrationが不十分であるという報告もあります。
参照url1: https://yukoishizaki.hatenablog.com/entry/2020/05/24/145155


/Users/ikegamikenshin/Documents/study/chatgpt-practice/.venv/lib/python3.9/site-packages/llama_index/data_structs/node_v2.py:181: UserWarning: .extra_info is deprecated, use .node.extra_info instead
  warnings.warn(".extra_info is deprecated, use .node.extra_info instead")


# Documentの追加

In [15]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10}

In [29]:
query_response = pinecone_index.query(
    top_k=10,
    include_metadata=True,
    # dummyのベクトル
    vector=[0.1] * 1536,
    filter={
         "extra_info_url": {"$in": [ADDITIONAL_URL]}
    }
)

# 既に登録されているURLの場合はskip
if len(query_response["matches"]) == 0:
    
    additional_urls = [ADDITIONAL_URL]
    additional_documents = SimpleWebPageReader(html_to_text=True).load_data(additional_urls)

    # extra_infoにurlを追加
    for additional_document, url in zip(additional_documents, additional_urls):
        additional_document.extra_info = {"url": url}
    
    # insert
    for additional_document in additional_documents:
        index.insert(additional_document)

INFO:llama_index.token_counter.token_counter:> [insert] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [insert] Total embedding token usage: 13970 tokens


In [30]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 14}},
 'total_vector_count': 14}

In [31]:
search_query = "Qdrantの特徴は?"

response = index.query(
    search_query,
    similarity_top_k=3,
    text_qa_template=CUSTOM_TEXT_QA_PROMPT,
    refine_template=CUSTOM_REFINE_PROMPT
)

print(f"response: {response.response}")

reffer_urls = set([source_node.extra_info["url"] for source_node in response.source_nodes])
for i, url in enumerate(reffer_urls):
    print(f"参照url{i+1}: {url}")

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 15872 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 10 tokens


response: Qdrantの特徴は、オープンソースのRust製ベクトル検索エンジンであり、Python SDK、REST API、gRPCで接続できること、クラウドサービス版も準備中であること、ベクトル検索においてコサイン類似度を使用していること、他のベクトル検索エンジンと比較して高速であることなどが挙げられます。Qdrantは、機械学習関連のブログでよく使用されるlivedoorニュースコーパスなどのベクトルデータを取り込むことができます。また、Qdrantの他にも、Facebook FaissやPynndescentなどのライブラリもベクトル検索に使用できます。Qdrantは、コレクションとポイントの概念を持ち、コレクションはRDBのテーブルに、ポイントはRDBのレコードに相当します。Qdrantでは、ペイロードと呼ばれるメタ情報も一緒に登録でき、フィルター検索に使用できます。Python SDKを使用することで、コレクションの作成、ドキュメントの登録、類似ドキュメントの検索が可能です。また、Qdrantはベクトル検索した後でフィルタリングすることができるため、検索結果が少なくなることを防ぐことができます。さらに、Qdrantはバッチクエリーという機能を持ち、一度に複数のクエリーを投げることができます。また、レコメンドAPIを使用することで、インデックスされたポイントを使用して検索できます。
参照url1: https://zenn.dev/tfutada/articles/acf8adbb2ba5be


In [32]:
# indexをローカルに保存
index.save_to_disk("../data/pinecone_index.json")

# index insertエラー時

In [7]:
pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENVIRONMENT"])
pinecone_index = pinecone.Index(INDEX_NAME)

In [9]:
llm_predictor = LLMPredictor(
    llm=OpenAI(
        temperature=0, model_name=PREDICTOR_MODEL_NAME, openai_api_key=os.environ["OPENAI_API_KEY"]
    )
)

embed_model = OpenAIEmbedding(
    model=EMBEDDING_MODEL_NAME, openai_api_key=os.environ["OPENAI_API_KEY"]
)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embed_model
)

index = GPTPineconeIndex.from_documents(
    pinecone_index=pinecone_index,
    documents=[], service_context=service_context
)

TypeError: __init__() got an unexpected keyword argument 'openai_api_key'

In [16]:
additional_urls = ["https://note.com/npaka/n/n2e2cf5a458ac"]
additional_documents = SimpleWebPageReader(html_to_text=True).load_data(additional_urls)

# extra_infoにurlを追加
for additional_document, url in zip(additional_documents, additional_urls):
    additional_document.extra_info = {"url": url}
    index.insert(additional_document)

ValueError: Encountered text corresponding to disallowed special token '<|endoftext|>'.
If you want this text to be encoded as a special token, pass it to `allowed_special`, e.g. `allowed_special={'<|endoftext|>', ...}`.
If you want this text to be encoded as normal text, disable the check for this token by passing `disallowed_special=(enc.special_tokens_set - {'<|endoftext|>'})`.
To disable this check for all special tokens, pass `disallowed_special=()`.


In [6]:
from llama_index import OpenAIEmbeddingModelType

ImportError: cannot import name 'OpenAIEmbeddingModelType' from 'llama_index' (/Users/ikegamikenshin/Documents/study/chatgpt-practice/.venv/lib/python3.9/site-packages/llama_index/__init__.py)